# Motor Control Bench Test Procedure

This notebook describes a procedure that exercises the following functionality:

1. Tachometers.
2. Arm/disarm and manual motor control.
3. Speed measurement.
4. PID motor control.

In [1]:
%load_ext autoreload
%autoreload 2

## Vehicle part identification and orientation conventions

![img](./Assets/RoverInTestBenchTopView.jpeg)

### Connection to vehicle and health check

Find which port is the vehicle connected to.
In linux, usually it is:

- **Development mode (STM32 UART3 / USB)**: ttyACM0, ttyACM1, ...
- **Production mode (STM32 UART 1)**: ttyUSB0, ttyUSB1, ...

Development/production mode must match the UartTcTmHandle parameter chosen when building firmware:

~~~c++
/* USER CODE END Header_DefaultTaskMain */
void DefaultTaskMain(void *argument)
{
  /* USER CODE BEGIN 5 */
	Config.UartTcTmHandle = &huart3; // Development mode
	//Config.UartTcTmHandle = &huart1; // Production mode
    
    /* ... */
    
	ApplicationMain(&Config);
  /* USER CODE END 5 */
}
~~~

In [ ]:
!ls /dev/ttyUSB*

In [2]:
!ls /dev/ttyACM*

/dev/ttyACM0


In [3]:
import sys
sys.path.append("../")
from rover.vehicleif import VehicleIF

#VEHICLE_UART_TCTM_PORT = "/dev/ttyUSB1" # Production mode
VEHICLE_UART_TCTM_PORT = "/dev/ttyACM0" # Development mode

connection_params = {
    "mode": VehicleIF.MODE_DIRECT_SERIAL,
    "port": VEHICLE_UART_TCTM_PORT,
    "baudrate": 115200
}

vehicle = VehicleIF(connection_params,debug=False)
vehicle.capture_path = "./captures"

For a quick health check, see current telemetry state:

In [4]:
vehicle.print_telemetry()

General Telemetry
   TelemetryCycle: 70
   OnBoardTime: 42004
   ReceivedPackets: 0
   GeneralStatus: 0x00000000
   Debug1: 0x00000000
   Debug2: 0x00000000
Motor Telemetry
   TelemetryCycle: 70
   OnBoardTime: 42004
   Throttle1: 0.0
   Throttle2: 0.0
   Tachometer1: 0
   Tachometer2: 0
   Tachometer3: 0
   Tachometer4: 0
   MeasuredSpeed1: 0.0
   MeasuredSpeed2: 0.0
   MeasuredSpeed3: 0.0
   MeasuredSpeed4: 0.0
   SetpointSpeed1: 0.0
   SetpointSpeed2: 0.0
   StatusFlags: 0x00000000
IMU Telemetry
   TelemetryCycle: 0
   OnBoardTime: 0
   Roll: 0.0
   Pitch: 0.0
   Yaw: 0.0


For a quick command test, turn on and off led.

In [5]:
# Led on
vehicle.control_led(0x00000001)

Sent: 0x 40 3C 05 00 00 00 00 01 A2 0A  (10 bytes)


In [6]:
# Led off
vehicle.control_led(0x00000000)

Sent: 0x 40 3C 05 00 00 00 00 00 93 0A  (10 bytes)


## Procedure

### 1. Tachometer readings

**Preconditions**

- Reset the vehicle to ensure all tachometer readings are at zero.

In [7]:
vehicle.print_telemetry()

General Telemetry
   TelemetryCycle: 87
   OnBoardTime: 52204
   ReceivedPackets: 2
   GeneralStatus: 0x01000000
   Debug1: 0x00000000
   Debug2: 0x00000000
Motor Telemetry
   TelemetryCycle: 87
   OnBoardTime: 52204
   Throttle1: 0.0
   Throttle2: 0.0
   Tachometer1: 0
   Tachometer2: 0
   Tachometer3: 0
   Tachometer4: 0
   MeasuredSpeed1: 0.0
   MeasuredSpeed2: 0.0
   MeasuredSpeed3: 0.0
   MeasuredSpeed4: 0.0
   SetpointSpeed1: 0.0
   SetpointSpeed2: 0.0
   StatusFlags: 0x00000000
IMU Telemetry
   TelemetryCycle: 0
   OnBoardTime: 0
   Roll: 0.0
   Pitch: 0.0
   Yaw: 0.0


Start logging telemetry.

In [ ]:
vehicle.start_telemetry_csv_logging()

In [ ]:
input("Move wheels and press any key when done")

- Move wheel 1.
- Move wheel 2.
- Move wheel 3.
- Move wheel 4.

Stop logging telemetry.

In [ ]:
vehicle.stop_telemetry_csv_logging()

In [ ]:
!ls {vehicle.capture_path}

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

motor_control_tm_df = pd.read_csv(os.path.join(vehicle.capture_path,"tctm.telemetry.MotorControlTelemetry.csv"))
motor_control_tm_df.head(5)

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(18,4))
axes.plot(motor_control_tm_df.Tachometer1)
axes.plot(motor_control_tm_df.Tachometer2)
axes.plot(motor_control_tm_df.Tachometer3)
axes.plot(motor_control_tm_df.Tachometer4)
axes.grid(which="Both")
axes.set_xlabel("Report index")
axes.set_ylabel("Tachometer tick count")
axes.legend(["Tachometer 1","Tachometer 2","Tachometer 3","Tachometer 4"])
axes.set_title("Tachometer readings");

### 2. Manual motor control

In [8]:
vehicle.print_telemetry()

General Telemetry
   TelemetryCycle: 186
   OnBoardTime: 111604
   ReceivedPackets: 2
   GeneralStatus: 0x01000000
   Debug1: 0x00000000
   Debug2: 0x00000000
Motor Telemetry
   TelemetryCycle: 186
   OnBoardTime: 111604
   Throttle1: 0.0
   Throttle2: 0.0
   Tachometer1: 0
   Tachometer2: 0
   Tachometer3: 0
   Tachometer4: 0
   MeasuredSpeed1: 0.0
   MeasuredSpeed2: 0.0
   MeasuredSpeed3: 0.0
   MeasuredSpeed4: 0.0
   SetpointSpeed1: 0.0
   SetpointSpeed2: 0.0
   StatusFlags: 0x00000000
IMU Telemetry
   TelemetryCycle: 0
   OnBoardTime: 0
   Roll: 0.0
   Pitch: 0.0
   Yaw: 0.0


Attempt to start motor when disarmed (should fail).

In [9]:
vehicle.set_motor_throttles(0.5,0.0,0x1)

Sent: 0x 40 3C 0D 01 00 00 00 01 3F 00 00 00 00 00 00 00 CC 0A  (18 bytes)


In [10]:
vehicle.print_telemetry()

General Telemetry
   TelemetryCycle: 197
   OnBoardTime: 118204
   ReceivedPackets: 3
   GeneralStatus: 0x00010000
   Debug1: 0x00000000
   Debug2: 0x00000000
Motor Telemetry
   TelemetryCycle: 196
   OnBoardTime: 117604
   Throttle1: 0.0
   Throttle2: 0.0
   Tachometer1: 0
   Tachometer2: 0
   Tachometer3: 0
   Tachometer4: 0
   MeasuredSpeed1: 0.0
   MeasuredSpeed2: 0.0
   MeasuredSpeed3: 0.0
   MeasuredSpeed4: 0.0
   SetpointSpeed1: 0.0
   SetpointSpeed2: 0.0
   StatusFlags: 0x00000000
IMU Telemetry
   TelemetryCycle: 0
   OnBoardTime: 0
   Roll: 0.0
   Pitch: 0.0
   Yaw: 0.0


Arm in manual mode.

In [11]:
vehicle.set_motor_control_mode(VehicleIF.MOTOR_MODE_ARM_MANUAL)

Sent: 0x 40 3C 05 03 00 00 00 01 E6 0A  (10 bytes)


In [12]:
vehicle.print_telemetry()

General Telemetry
   TelemetryCycle: 206
   OnBoardTime: 123604
   ReceivedPackets: 4
   GeneralStatus: 0x01030001
   Debug1: 0x00000000
   Debug2: 0x00000000
Motor Telemetry
   TelemetryCycle: 206
   OnBoardTime: 123604
   Throttle1: 0.0
   Throttle2: 0.0
   Tachometer1: 0
   Tachometer2: 0
   Tachometer3: 0
   Tachometer4: 0
   MeasuredSpeed1: 0.0
   MeasuredSpeed2: 0.0
   MeasuredSpeed3: 0.0
   MeasuredSpeed4: 0.0
   SetpointSpeed1: 0.0
   SetpointSpeed2: 0.0
   StatusFlags: 0x00000001
IMU Telemetry
   TelemetryCycle: 0
   OnBoardTime: 0
   Roll: 0.0
   Pitch: 0.0
   Yaw: 0.0


Check vehicle right side moves forward.

In [13]:
vehicle.set_motor_throttles(0.5,0.0,0x1)

Sent: 0x 40 3C 0D 01 00 00 00 01 3F 00 00 00 00 00 00 00 CC 0A  (18 bytes)


Check vehicle right side moves backward.

In [14]:
vehicle.set_motor_throttles(-0.5,0.0,0x1)

Sent: 0x 40 3C 0D 01 00 00 00 01 BF 00 00 00 00 00 00 00 0E 0A  (18 bytes)


Stop.

In [15]:
vehicle.set_motor_throttles(0.0,0.0,0x1)

Sent: 0x 40 3C 0D 01 00 00 00 01 00 00 00 00 00 00 00 00 D4 0A  (18 bytes)


Check vehicle left side moves backward.

In [16]:
vehicle.set_motor_throttles(0.0,0.5,0x2)

Sent: 0x 40 3C 0D 01 00 00 00 02 00 00 00 00 3F 00 00 00 FC 0A  (18 bytes)


Check vehicle left side moves backward.

In [17]:
vehicle.set_motor_throttles(0.0,-0.5,0x2)

Sent: 0x 40 3C 0D 01 00 00 00 02 00 00 00 00 BF 00 00 00 0D 0A  (18 bytes)


Both sides forward.

In [18]:
vehicle.set_motor_throttles(0.5,0.5,0x3)

Sent: 0x 40 3C 0D 01 00 00 00 03 3F 00 00 00 3F 00 00 00 51 0A  (18 bytes)


Both sides backward.

In [19]:
vehicle.set_motor_throttles(-0.5,-0.5,0x3)

Sent: 0x 40 3C 0D 01 00 00 00 03 BF 00 00 00 BF 00 00 00 62 0A  (18 bytes)


Disarm.

In [20]:
vehicle.set_motor_control_mode(VehicleIF.MOTOR_MODE_DISARM)

Sent: 0x 40 3C 05 03 00 00 00 00 D7 0A  (10 bytes)


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/nhorro/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/nhorro/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "../rover/endpoint/serialif.py", line 36, in _listen
    if self.serial and self.serial.is_open and self.serial.in_waiting:
  File "/home/nhorro/anaconda3/lib/python3.8/site-packages/serial/serialposix.py", line 549, in in_waiting
    s = fcntl.ioctl(self.fd, TIOCINQ, TIOCM_zero_str)
OSError: [Errno 5] Input/output error


### 3. Speed measurement

In [ ]:
import numpy as np
import time

In [ ]:
MIN_ABS_THROTTLE=0.0
MAX_ABS_THROTTLE=1.0
N_SAMPLES = 32
INTERVAL_IN_S = 1.0
SUSTAIN_INTERVAL_IN_S = 5.0

In [ ]:
def do_motor_curve(vehicle, min_value, max_value, n_samples, interval, side):
    for throttle in np.linspace(min_value,max_value,n_samples):
        if side == 'left':
            vehicle.set_motor_throttles(throttle,0.0,0x1)
        elif side == 'right':
            vehicle.set_motor_throttles(0.0, throttle, 0x2)
        else:
            vehicle.set_motor_throttles(throttle,throttle,0x3)
        time.sleep(interval)

In [ ]:
def perform_motor_test(vehicle, min_abs_throttle, max_abs_throttle, n_samples, interval, sustain_interval, side):
    # Start logging
    vehicle.start_telemetry_csv_logging()

    # Arm 
    vehicle.set_motor_control_mode(VehicleIF.MOTOR_MODE_ARM_MANUAL)
    time.sleep(0.5)

    # Ramp up
    do_motor_curve(vehicle, min_abs_throttle, max_abs_throttle, n_samples, interval, side)
    # Sustain
    time.sleep(sustain_interval)
    # Ramp down
    do_motor_curve(vehicle, max_abs_throttle, min_abs_throttle, n_samples, interval, side)

    # Disarm
    vehicle.set_motor_control_mode(VehicleIF.MOTOR_MODE_DISARM)

    # Stop logging
    vehicle.stop_telemetry_csv_logging()

#### 3.1 Left side

In [ ]:
SIDE = "left" # left, right, or both
perform_motor_test(vehicle,MIN_ABS_THROTTLE,MAX_ABS_THROTTLE,N_SAMPLES,INTERVAL_IN_S,SUSTAIN_INTERVAL_IN_S,SIDE)

In [ ]:
!cp {vehicle.capture_path}/tctm.telemetry.MotorControlTelemetry.csv {vehicle.capture_path}/tctm.telemetry.MotorControlTelemetry-LeftThrottleRamp.csv

In [ ]:
motor_control_tm_df = pd.read_csv(os.path.join(vehicle.capture_path,"tctm.telemetry.MotorControlTelemetry-LeftThrottleRamp.csv"))
motor_control_tm_df["AverageSpeed12"] = (motor_control_tm_df.MeasuredSpeed1+ motor_control_tm_df.MeasuredSpeed2)/2
motor_control_tm_df.tail()

In [ ]:
fig, axes = plt.subplots(3,1,figsize=(18,12),sharex=True)
axes[0].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Throttle1)
axes[0].set_title("Throttle")
axes[0].grid(which="Both")
axes[0].set_xlabel("Time [s]")
axes[0].set_ylabel("Throttle [%]")
axes[0].legend(["Throttle1"])

axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.MeasuredSpeed1)
axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.MeasuredSpeed2)
axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.AverageSpeed12)
axes[1].set_title("Measured speed(both wheels)")
axes[1].grid(which="Both")
axes[1].set_xlabel("Time [s]")
axes[1].set_ylabel("Measured speed [RPM]")
axes[1].legend(["Speed1","Speed2","Average"])

axes[2].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Tachometer1)
axes[2].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Tachometer2)
axes[2].grid(which="Both")
axes[2].set_xlabel("Time [s]")
axes[2].set_ylabel("Tachometer tick count")
axes[2].legend(["Tachometer 1","Tachometer 2"])
axes[2].set_title("Tachometer readings");

#### 3.2 Right side

In [ ]:
SIDE = "right" # left, right, or both
perform_motor_test(vehicle,MIN_ABS_THROTTLE,MAX_ABS_THROTTLE,N_SAMPLES,INTERVAL_IN_S,SUSTAIN_INTERVAL_IN_S,SIDE)

In [ ]:
!cp {vehicle.capture_path}/tctm.telemetry.MotorControlTelemetry.csv {vehicle.capture_path}/tctm.telemetry.MotorControlTelemetry-RightThrottleRamp.csv

In [ ]:
motor_control_tm_df = pd.read_csv(os.path.join(vehicle.capture_path,"tctm.telemetry.MotorControlTelemetry-RightThrottleRamp.csv"))
motor_control_tm_df["AverageSpeed34"] = (motor_control_tm_df.MeasuredSpeed3+ motor_control_tm_df.MeasuredSpeed4)/2
motor_control_tm_df.tail()

In [ ]:
fig, axes = plt.subplots(3,1,figsize=(18,16),sharex=True)
axes[0].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Throttle2)
axes[0].set_title("Throttle")
axes[0].grid(which="Both")
axes[0].set_xlabel("Time [s]")
axes[0].set_ylabel("Throttle [%]")
axes[0].legend(["Throttle1"])

axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.MeasuredSpeed3)
axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.MeasuredSpeed4)
axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.AverageSpeed34)
axes[1].set_title("Measured speed(both wheels)")
axes[1].grid(which="Both")
axes[1].set_xlabel("Time [s]")
axes[1].set_ylabel("Measured speed [RPM]")
axes[1].legend(["Speed3","Speed4","Average"])

axes[2].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Tachometer3)
axes[2].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Tachometer4)
axes[2].grid(which="Both")
axes[2].set_xlabel("Time [s]")
axes[2].set_ylabel("Tachometer tick count")
axes[2].legend(["Tachometer 3","Tachometer 4"])
axes[2].set_title("Tachometer readings");

#### 3.3 Both sides

In [ ]:
SIDE = "both" # left, right, or both
perform_motor_test(vehicle,MIN_ABS_THROTTLE,MAX_ABS_THROTTLE,N_SAMPLES,INTERVAL_IN_S,SUSTAIN_INTERVAL_IN_S,SIDE)

In [ ]:
!cp {vehicle.capture_path}/tctm.telemetry.MotorControlTelemetry.csv {vehicle.capture_path}/tctm.telemetry.MotorControlTelemetry-BothThrottleRamp.csv

In [ ]:
motor_control_tm_df = pd.read_csv(os.path.join(vehicle.capture_path,"tctm.telemetry.MotorControlTelemetry-BothThrottleRamp.csv"))
motor_control_tm_df["AverageSpeed12"] = (motor_control_tm_df.MeasuredSpeed1+ motor_control_tm_df.MeasuredSpeed2)/2
motor_control_tm_df["AverageSpeed34"] = (motor_control_tm_df.MeasuredSpeed3+ motor_control_tm_df.MeasuredSpeed4)/2
motor_control_tm_df.tail()

In [ ]:
fig, axes = plt.subplots(4,1,figsize=(18,16),sharex=True)
axes[0].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Throttle1)
axes[0].set_title("Throttle")
axes[0].grid(which="Both")
axes[0].set_xlabel("Time [s]")
axes[0].set_ylabel("Throttle [%]")
axes[0].legend(["Throttle1"])

axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.MeasuredSpeed1)
axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.MeasuredSpeed2)
axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.MeasuredSpeed3)
axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.MeasuredSpeed4)
axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.AverageSpeed12)
axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.AverageSpeed34)
axes[1].set_title("Measured speed (all wheels)")
axes[1].grid(which="Both")
axes[1].set_xlabel("Time [s]")
axes[1].set_ylabel("Measured speed [RPM]")
axes[1].legend(["Speed1","Speed2","Speed3","Speed4","Average12","Average34",])

axes[2].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.AverageSpeed12.rolling(4).mean())
axes[2].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.AverageSpeed34.rolling(4).mean())
axes[2].set_title("Measured speed(both wheels)")
axes[2].grid(which="Both")
axes[2].set_xlabel("Time [s]")
axes[2].set_ylabel("Measured speed [RPM]")
axes[2].legend(["Average (filtered)"])

axes[3].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Tachometer1)
axes[3].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Tachometer2)
axes[3].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Tachometer3)
axes[3].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Tachometer4)
axes[3].grid(which="Both")
axes[3].set_xlabel("Time [s]")
axes[3].set_ylabel("Tachometer tick count")
axes[3].legend(["Tachometer 1","Tachometer 2","Tachometer 3","Tachometer 4"])
axes[3].set_title("Tachometer readings");

### 4. PID motor control

In [ ]:
vehicle.print_telemetry()

In [ ]:
vehicle.set_pid_parameters(0.01,1,1)

In [ ]:
# Start logging
vehicle.start_telemetry_csv_logging()

In [ ]:
MIN_SPEED = 40
MAX_SPEED = 400
N_SAMPLES = 64
INTERVAL=0.1

# Start logging
vehicle.start_telemetry_csv_logging()

vehicle.set_motor_control_mode(VehicleIF.MOTOR_MODE_ARM_PID)
time.sleep(1)

for speed in np.linspace(MIN_SPEED,MAX_SPEED,N_SAMPLES):
    vehicle.set_motor_speeds(speed,speed,0x3)
    time.sleep(INTERVAL)

# Sustain
time.sleep(INTERVAL*50)


for speed in np.linspace(MAX_SPEED,MIN_SPEED,N_SAMPLES):
    vehicle.set_motor_speeds(speed,speed,0x3)
    time.sleep(INTERVAL)

time.sleep(1)

# Stop logging
vehicle.stop_telemetry_csv_logging()     

# Disarm
vehicle.set_motor_control_mode(VehicleIF.MOTOR_MODE_DISARM)

In [ ]:
motor_control_tm_df = pd.read_csv(os.path.join(vehicle.capture_path,"tctm.telemetry.MotorControlTelemetry.csv"))
motor_control_tm_df["AverageSpeed12"] = (motor_control_tm_df.MeasuredSpeed1+ motor_control_tm_df.MeasuredSpeed2)/2
motor_control_tm_df.tail()

In [ ]:
fig, axes = plt.subplots(4,1,figsize=(18,16),sharex=True)
axes[0].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Throttle1)
axes[0].set_title("Throttle")
axes[0].grid(which="Both")
axes[0].set_xlabel("Time [s]")
axes[0].set_ylabel("Throttle [%]")
axes[0].legend(["Throttle1"])

axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.MeasuredSpeed1)
axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.MeasuredSpeed2)
axes[1].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.AverageSpeed12)
axes[1].set_title("Measured speed (all wheels)")
axes[1].grid(which="Both")
axes[1].set_xlabel("Time [s]")
axes[1].set_ylabel("Measured speed [RPM]")
axes[1].legend(["Speed1","Speed2","Average"])

axes[2].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.AverageSpeed12.rolling(4).mean())
axes[2].set_title("Measured speed(both wheels)")
axes[2].grid(which="Both")
axes[2].set_xlabel("Time [s]")
axes[2].set_ylabel("Measured speed [RPM]")
axes[2].legend(["Average (filtered)"])

axes[3].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Tachometer1)
axes[3].plot(motor_control_tm_df.OnBoardTime-motor_control_tm_df.OnBoardTime.iloc[0]/1000,motor_control_tm_df.Tachometer2)
axes[3].grid(which="Both")
axes[3].set_xlabel("Time [s]")
axes[3].set_ylabel("Tachometer tick count")
axes[3].legend(["Tachometer 1","Tachometer 2",])
axes[3].set_title("Tachometer readings");

## Teardown

In [ ]:
vehicle.shutdown()

# Debug
<hr/>

In [ ]:
vehicle.set_motor_control_mode(VehicleIF.MOTOR_MODE_DISARM)

In [ ]:
vehicle.set_motor_control_mode(VehicleIF.MOTOR_MODE_ARM_MANUAL)
vehicle.set_motor_throttles(0.4,0.,0x1)

In [ ]:
vehicle.print_telemetry()